In [4]:
!pip install surprise


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
from surprise import Reader, Dataset, SVDpp
import random

user_data = pd.read_csv('https://raw.githubusercontent.com/CloudComputing20/mosymovie_ai/main/userdata.csv', encoding='CP949')
movie_data = pd.read_csv('https://raw.githubusercontent.com/CloudComputing20/mosymovie_ai/main/moviedata.csv', encoding='CP949')
input_data = pd.read_csv('https://raw.githubusercontent.com/CloudComputing20/mosymovie_ai/main/nowPlayingMovie_1.csv', encoding='CP949')

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(user_data[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()

algo = SVDpp(n_factors=100, n_epochs=20, random_state=42)
algo.fit(trainset)

max_movie_id = movie_data['movieId'].max()

for index, row in input_data.iterrows():
    movie_name = row['title']
    movie_genre = row['genres'].split('|')

    new_movie_id = max_movie_id + 1
    max_movie_id = new_movie_id
    movie_data.loc[len(movie_data)] = [new_movie_id, movie_name, '|'.join(movie_genre)]

    movie_id = new_movie_id

    user_list = []
    for user_id in user_data['userId'].unique():
        user_ratings = user_data[user_data['userId'] == user_id]
        user_movies = set(user_ratings['movieId'])
        if movie_id not in user_movies:
            user_avg_rating = user_ratings['rating'].mean()

            est_rating = algo.predict(uid=user_id, iid=movie_id, r_ui=None, verbose=False).est * random.uniform(0.9, 1.1)
            if est_rating >= user_avg_rating or est_rating >= 4:
                user_list.append(user_id)

    user_list.insert(0, movie_name)
    print(user_list)

    movie_data.drop(movie_data.tail(1).index, inplace=True)

KeyboardInterrupt: 